In [ ]:
# Install required packages
%pip install -r requirements.txt


In [1]:
# Import core dependencies
import os
import asyncio
import json
from datetime import datetime, timedelta

# Strands imports
from strands import Agent
from strands.models.bedrock import BedrockModel
from strands.tools.mcp import MCPClient
from strands_tools import current_time, web_search

# Project imports
from utils.config import Config
from utils.observability import setup_langfuse_tracing
from agents.orchestrator import MultiAgentOrchestrator
from agents.research_agent import create_research_agent
from agents.analysis_agent import create_analysis_agent
from agents.report_agent import create_report_agent

print("✅ All imports successful")


ImportError: cannot import name 'web_search' from 'strands_tools' (/opt/homebrew/Caskroom/miniforge/base/envs/genai-on-aws/lib/python3.12/site-packages/strands_tools/__init__.py)

In [ ]:
# Initialize the system
async def demo_weather_intelligence():
    """Run a complete weather intelligence demo."""
    
    # Setup configuration
    config = Config()
    print("🔧 Configuration loaded")
    
    # Setup observability (optional - will gracefully fail if credentials not set)
    try:
        setup_langfuse_tracing(
            public_key=config.LANGFUSE_PUBLIC_KEY or "demo",
            secret_key=config.LANGFUSE_SECRET_KEY or "demo", 
            host=config.LANGFUSE_HOST
        )
        print("✅ LangFuse observability configured")
    except Exception as e:
        print(f"⚠️ LangFuse setup skipped: {e}")
    
    # Create agents
    print("🤖 Creating specialized agents...")
    research_agent = await create_research_agent()
    analysis_agent = await create_analysis_agent()
    report_agent = await create_report_agent()
    print("✅ Agents created successfully")
    
    # Create orchestrator
    orchestrator = MultiAgentOrchestrator(
        research_agent=research_agent,
        analysis_agent=analysis_agent, 
        report_agent=report_agent,
        coordination_pattern="mesh"
    )
    print("🎭 Multi-agent orchestrator ready")
    
    # Demo task
    task = """
    Analyze weather conditions affecting corn production in Iowa over the next week.
    Consider:
    1. Current weather patterns and 7-day forecast
    2. Impact on corn growth stages
    3. Risk assessment for farmers
    4. Recommendations for mitigation
    
    Provide a comprehensive report for agricultural stakeholders.
    """
    
    print("\n🌽 Executing: Iowa Corn Weather Impact Analysis")
    print("=" * 60)
    
    # Execute the multi-agent task
    result = await orchestrator.execute_task(task)
    
    print(f"\n📊 Analysis Complete!")
    print(f"Duration: {result.total_duration:.1f} seconds")
    print(f"Coordination: {result.coordination_pattern}")
    print(f"Success: {result.success}")
    
    print(f"\n📋 Final Report:")
    print("=" * 60)
    print(result.final_report)
    print("=" * 60)
    
    return result

# Run the demo
result = await demo_weather_intelligence()
